![](https://i.redd.it/nklty63uzav41.png)

# Introduction

In this notebook, let's get to know about the fastest way of reading and processing images. We will use iWildCam2021 dataset and compare various libraries available using a benchmarking task. A fast and efficient data loading pipeline will help us train our models faster and **save a lot of GPU time**.

Notebook is distributed in following sections -
1. Libraries for image processing
2. Create a benchmark task
3. Compare the libraries
4. Conclusion
5. References and further reading

<div style="color:#FFFFFF; background:#34b1eb; border-radius:5px;"> 
    <center><br> <h1>1. Libraries for image processing</h1> <br></center>
</div>

### The most famous ones - 
1. OpenCV
2. Scikit Image
3. Pillow
4. Pillow-SIMD

There is a lot of documentation available for all of them. I recommend reading basic things about all of them in case you are not aware.

In [ ]:
# OpenCV
import cv2

# Scikit-Image
import skimage
from skimage import io
from skimage.transform import resize

# Pillow/PIL
import PIL
from PIL import Image

# To use Pillow-simd you just need to unistall PIL and install PIL-simd

# Other helper libraries
import json
from matplotlib import pyplot as plt

In [ ]:
# Versions used
print("OpenCV: ", cv2.__version__)
print("Scikit Image: ", skimage.__version__)
print("PIL: ", PIL.__version__)

<div style="color:#FFFFFF; background:#34b1eb; border-radius:5px;"> 
    <center><br> <h1>2. Create a benchmark task</h1> <br></center>
</div>

<br>

We will ofcourse use [iWildCam2021](https://www.kaggle.com/c/iwildcam2021-fgvc8/overview) dataset. I recommend going through this wonderful [notebook](https://www.kaggle.com/nayuts/iwildcam-2021-starter-notebook) if you are not fully aware about the competition and dataset. 

The task will be simple but a little tiresome.

**The task is to read 500 images and for each image, crop the annotated detections (provided in json file) and then resize the same.**

1. For each image in 500 images <br>
    i. Read the image <br>
    ii. Crop the detections <br>
    iii. Resize the cropped detections <br>

This task will test Reading, Cropping and Resizing operations of all libraries which is core to any image preprocessing pipeline

In [ ]:
with open('../input/iwildcam2021-fgvc8/metadata/iwildcam2021_megadetector_results.json', encoding='utf-8') as json_file:
    detections = json.load(json_file)

# detections['images'][0] looks like
# {
#     'detections': [{'category': '1', 'bbox': [0.6529, 0.5425, 0.3471, 0.4038], 'conf': 0.999}], 
#     'id': '905a3c8c-21bc-11ea-a13a-137349068a90', 
#     'max_detection_conf': 0.999
# }

data = {} # {image_name:[detections]}

total_im_reads = 500
total_im_crops = 0
for detection in detections['images'][:total_im_reads]:
    data[detection['id']+ '.jpg'] = [x['bbox'] for x in detection['detections']]
    total_im_crops += len(detection['detections'])
    
print(f"The benchmarking task involves reading images {total_im_reads} times, cropping and resizing images {total_im_crops} times.")

We have created a dictionary {} called data which has names and detections of our images.

<div style="color:#FFFFFF; background:#34b1eb; border-radius:5px;"> 
    <center><br> <h1>3. Compare the libraries</h1> <br></center>
</div>

<br>

We will use %timeit module to measure the time. And every crop will be resized to 200 x 200 image size. 

In [ ]:
SIZE = 200

## 1. OpenCV

In [ ]:
def numpy_crop_image(img, bbox):
    h,w,c = img.shape
    x1, y1,w_box, h_box = bbox
    ymin,xmin,ymax,xmax = y1, x1, y1 + h_box, x1 + w_box
    ymin,xmin,ymax,xmax = ymin*h,xmin*w,ymax*h,xmax*w
    crop_img = img[int(ymin):int(ymax), int(xmin):int(xmax)]
    return crop_img

def opencv_task():
    for image, detections in data.items():
        img = cv2.imread("../input/iwildcam2021-fgvc8/train/"+image)
        for detection in detections:
            crop = numpy_crop_image(img, detection)
            resized = cv2.resize(crop,(SIZE,SIZE))
            results.append(resized)

In [ ]:
results = []
%timeit opencv_task()

In [ ]:
# See some results
fig = plt.figure(figsize=(25, 25))
for i,result in enumerate(results[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    ax.title.set_text(f'OpenCV {i}')
    plt.imshow(result)

## 2. Scikit-Image

OpenCV and Scikit-Image both work with numpy arrays. Thus, both use numpy array slicing for cropping images. And hence we will use extact same function for both of these.

In [ ]:
def skimage_task():
    for image, detections in data.items():
        img = io.imread("../input/iwildcam2021-fgvc8/train/"+image)
        for detection in detections:
            crop = numpy_crop_image(img, detection)
            resized = resize(crop,(SIZE,SIZE))
            results.append(resized)

In [ ]:
results = []
%timeit skimage_task()

In [ ]:
# See some results
fig = plt.figure(figsize=(25, 25))
for i,result in enumerate(results[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    ax.title.set_text(f'Scikit-Image {i}')
    plt.imshow(result)

## 3. PIL

In [ ]:
def pil_crop_image(bbox, image_size):
    x1, y1,w_box, h_box = bbox
    ymin,xmin,ymax, xmax = y1, x1, y1 + h_box, x1 + w_box
    area = (xmin * image_size[0], ymin * image_size[1], 
            xmax * image_size[0], ymax * image_size[1])
    return area

def pil_task():
    for image, detections in data.items():
        img = Image.open("../input/iwildcam2021-fgvc8/train/"+image)
        for detection in detections:
            area = pil_crop_image(detection, img.size)
            crop = img.crop(area)
            resized = crop.resize((SIZE,SIZE))
            results.append(resized)

In [ ]:
results = []
%timeit pil_task()

In [ ]:
# See some results
fig = plt.figure(figsize=(25, 25))
for i,result in enumerate(results[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    ax.title.set_text(f'PIL {i}')
    plt.imshow(result)

## 4. Pillow-simd

<br>

To use pillow-simd you just need to unistall pillow and install pillow-simd. And it works exactly same as normal PIL.

In [ ]:
!pip uninstall -y pillow
!pip install pillow-simd

In [ ]:
results = []
%timeit pil_task()

In [ ]:
# See some results
fig = plt.figure(figsize=(25, 25))
for i,result in enumerate(results[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    ax.title.set_text(f'PIL-SIMD {i}')
    plt.imshow(result)

<div style="color:#FFFFFF; background:#34b1eb; border-radius:5px;"> 
    <center><br> <h1>4. Conclusion</h1> <br></center>
</div>

<br>

### Winner : PIL 🔥🔥🔥

<br>

### Comparison :
<br>

We ran each method of the same task 7 times and following is the average time for one loop. Width of each bar = (2 x (mean_time x 10)) pixels
<div style="padding:10px;width:350px;background:#ffadad;"><center>Opencv: 17.5 s ± 66 ms</center></div>
<div style="padding:10px;width:982px;background:#e1ffad"><center>Skimage: 49.1 s ± 2.26 s</center></div>
<div style="padding:10px;width:230px;background:#adffdb"><center>PIL: 11.5 s ± 33.2 ms</center></div>
<div style="padding:10px;width:232px;background:#d5adff"><center>PIL-simd: 11.6 s ± 53.1 ms</center></div>

<br>

Even if PIL-SIMD is highly optimized library, we see that PIL performs slightly better. 

**Only a very detailed benchmarking would reveal strengths and weaknesses of each library. But as of now, to do the basic reading, cropping and resizing we see that PIL is the fastest.**

<div style="color:#FFFFFF; background:#34b1eb; border-radius:5px;"> 
    <center><br> <h1>5. References and further reading</h1> <br></center>
</div>

<br>

https://learnopencv.com/efficient-image-loading/

https://python-pillow.org/pillow-perf/

https://www.kaggle.com/vfdev5/pil-vs-opencv

<center>
    <div style="color:red;">
        <h2>Please don't forget to upvote if you find this notebook useful :)</h2>
    </div>
<center>